In [2]:
import numpy as np
import pandas as pd

## 1b)

In [3]:
data = pd.read_csv('C:/Users/ldmag/Downloads/logit_ridge.csv', header=None)

In [4]:
# add ones for intercept; apparently you need to do this
#data = np.column_stack((np.ones(len(data)), data))

In [5]:
test_set = data[0:10]
training_set = data[10:]

In [6]:
x_train = training_set.loc[:,1:21].values
y_train = training_set.loc[:,:0].values

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(penalty='l2', C=1, fit_intercept=True) # Applying ridge penalty with lambda = 1 and intercept
model.fit(x_train, y_train.ravel())

LogisticRegression(C=1)

In [7]:
# print intercepts and coefficients (betas)
print(model.intercept_)
print('Value of Beta 1:', model.coef_[0,0])
print('Value of Beta 2:', model.coef_[0,1])

[-0.14655573]
Value of Beta 1: -2.6938423917371095
Value of Beta 2: -0.2847969442088175


From scratch:

In [53]:
def has_converged(beta_old, beta_new, tol=1e-8):
    return np.all(np.abs(beta_new - beta_old) < tol)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def calculate_likelihood(X, y, beta):
    y_pred = sigmoid(np.dot(X, beta))
    likelihood = np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
    return likelihood

def calculate_penalty(beta):
    penalty = np.sum(beta**2)
    return penalty

def calculate_objective(X, y, beta, _lambda):
    likelihood = calculate_likelihood(X, y, beta)
    penalty = _lambda * calculate_penalty(beta)
    return likelihood - penalty

def gradient_ascent(X, y, _lambda, alpha, num_iterations=50000, tol=1e-8):
    # Normalize the features; using LogReg function documentation as reference from sklearn
    X_normalized = (X - X.mean(axis=0)) / X.std(axis=0)

    # Add a column of ones to X for the intercept term
    X_normalized = np.column_stack((np.ones(len(X_normalized)), X_normalized))

    num_features = X_normalized.shape[1]
    beta = np.zeros((num_features, 1))

    for i in range(num_iterations):
        # Store the old beta values for convergence check
        beta_old = beta.copy()

        # Calculate the predicted values using the sigmoid function
        y_pred = sigmoid(np.dot(X_normalized, beta))

        # Calculate the gradient of the objective function
        error = y - y_pred
        likelihood_gradient = -np.dot(X_normalized.T, error)
        penalty_gradient = 2 * _lambda * beta
        gradient = likelihood_gradient + penalty_gradient

        # Update beta using a smaller step size
        beta = beta + alpha * gradient

        # Print the objective value and beta values for every 5000 iterations

        # Check for convergence
        if has_converged(beta_old, beta, tol):
            print("Converged! Stopping optimization.")
            break

    return beta.flatten()

In [54]:
beta_estimate = gradient_ascent(x_train, y_train, _lambda=1, alpha=1e-4, num_iterations=50)

In [56]:
print('Beta 1 estimate:', beta_estimate[1])
print('Beta 2 estimate:', beta_estimate[2])
print('Intercept:', beta_estimate[0])

Beta 1 estimate: 0.15085753165723068
Beta 2 estimate: 0.03755128426486737
Intercept: 0.09559361227620934


## 1c)

In [57]:
x_test = test_set.loc[:,1:20].values
y_test = test_set.loc[:,:0].values

In [58]:
y_pred = model.predict(x_test)
errors = (y_test - y_pred)**2 #euclidean distance of error

prediction_df = pd.DataFrame({'Actual': y_test.flatten(),
                              'Predicted': y_pred})

In [59]:
y_pred_probabilities = model.predict_proba(x_test)[:,1]
errors = (y_test - y_pred_probabilities)**2

In [60]:
print('Average test error:', errors.mean())

Average test error: 0.36793528837466366


From scratch

In [61]:
x_test_norm = (x_test - x_train.mean(axis=0)) / x_train.std(axis=0)
# add intercept
x_test_norm = np.column_stack((np.ones(len(x_test_norm)), x_test_norm))

# predictions
y_pred = sigmoid(np.dot(x_test_norm, beta_estimate))
test_errors = np.abs(y_test-y_pred)
print('Average Test Error:', np.mean(test_errors))

Average Test Error: 0.506510762111126
